# [IAPR][iapr]: Final project - Chocolate Recognition


**Moodle group ID:** *xx*  
**Kaggle challenge:** *xx* (either `Classic` or `Deep learning`)  
**Kaggle team name (exact):** "*xx*"  

**Author 1 (SCIPER):** *Student Name 1 (xxxxx)*  
**Author 2 (SCIPER):** *Student Name 2 (xxxxx)*  
**Author 3 (SCIPER):** *Student Name 3 (xxxxx)*  

**Due date:** 21.05.2025 (11:59 pm)


## Key Submission Guidelines:
- **Before submitting your notebook, <span style="color:red;">rerun</span> it from scratch!** Go to: `Kernel` > `Restart & Run All`
- **Only groups of three will be accepted**, except in exceptional circumstances.


[iapr]: https://github.com/LTS5/iapr2025

---

## Data augmentation

In [7]:
# 数据增强：chocolate_augmentation.ipynb

# 第 1 格：导入必要库
import os
import cv2
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

# 第 2 格：路径设置
INPUT_DIR = 'D:/exchange/EE-451/Choco/EE-451-project/dataset_project_iapr2025/train'
OUTPUT_DIR = 'dataset_project_iapr2025/train_aug'
CSV_PATH = 'D:/exchange/EE-451/Choco/EE-451-project/dataset_project_iapr2025/train.csv'
OUTPUT_CSV_PATH = 'dataset_project_iapr2025/train_augmented.csv'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 第 3 格：读取标签数据
df = pd.read_csv(CSV_PATH)

# 第 4 格：定义增强函数（使用 OpenCV 和 NumPy）
def apply_augmentation(img):
    h, w = img.shape[:2]
    
    # 随机旋转
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    img = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)

    # 水平翻转
    if random.random() < 0.5:
        img = cv2.flip(img, 1)

    # 随机亮度/对比度调整
    alpha = random.uniform(0.8, 1.2)  # 对比度
    beta = random.uniform(-10, 10)    # 亮度
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # 添加高斯模糊（有概率）
    if random.random() < 0.3:
        ksize = random.choice([3, 5])
        img = cv2.GaussianBlur(img, (ksize, ksize), 0)

    return img

# 第 5 格：设置每张图像增强几次
AUG_TIMES = 20

# 第 6 格：执行增强
augmented_records = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    original_img_name =  f"L{int(row['id'])}.jpg"
    original_img_path = os.path.join(INPUT_DIR, original_img_name)

    image = cv2.imread(original_img_path)
    if image is None:
        continue

    # 添加原图记录（不拷贝图像，只复制标签）
    augmented_records.append(row)

    for i in range(AUG_TIMES):
        aug_img = apply_augmentation(image)
        new_name = f"aug_{i}_{original_img_name}"
        new_path = os.path.join(OUTPUT_DIR, f"L{str(new_name)}.jpg")
        cv2.imwrite(new_path, aug_img)

        new_row = row.copy()
        new_row['id'] = new_name
        augmented_records.append(new_row)

# 第 7 格：保存新的 CSV
aug_df = pd.DataFrame(augmented_records)
aug_df.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"数据增强完成：生成 {len(aug_df)} 条图像标签")


  0%|          | 0/90 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_29312\2101181021.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'aug_0_L1000756.jpg' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_row['id'] = new_name
C:\Users\User\AppData\Local\Temp\ipykernel_29312\2101181021.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'aug_1_L1000756.jpg' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_row['id'] = new_name
C:\Users\User\AppData\Local\Temp\ipykernel_29312\2101181021.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'aug_2_L1000756.jpg' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

KeyboardInterrupt: 